# Import Libraries

In [1]:
# Built-in
import warnings
import platform
from typing import Literal, Union, Optional, Dict, List, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# PyTorch
# import torch
# from torch import nn

# Models
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# Scikit-learn
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)

# Imbalanced-learn
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.combine import SMOTEENN, SMOTETomek

## 추가 설정
- 경고 메세지 무시
- Dataframe 출력시 모든 행과 열 표시
- 한글 폰트 설정
- SEED 설정 (SEED = 123)

In [2]:
# 경고 메시지 무시
warnings.filterwarnings("ignore")

# Dataframe 출력시 모든 행과 열 표시
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':
    plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# SEED 설정
SEED = 123

# 데이터 불러오기 및 전처리

## 전처리 순서 및 세부 사항
1. **Column 제거**
    - 결측 비율 80% 이상 / nunique = 1 / 배란 유도 유형, 특정 시술 유형, 남성 및 여성 주 부 불임 원인 col 제거
    - 제거된 column 개수: 16개
    - 제거된 column 목록: ['난자 해동 경과일', '배아 해동 경과일', 'PGD 시술 여부', '불임 원인 - 여성 요인', '난자 채취 경과일', '남성 주 불임 원인', '부부 주 불임 원인', 'PGS 시술 여부', '임신 시도 또는 마지막 임신 경과 연수', '부부 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '특정 시술 유형', '남성 부 불임 원인', '배란 유도 유형', '착상 전 유전 검사 사용 여부']

2. **int 변환**
    - 횟수가 포함된 column int 변환
    - 변환된 column 개수: 10개
    - 변환된 column 목록: ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']
3. **시술 횟수 재정의**
    - 총 시술 횟수 = IVF 시술 횟수 + DI 시술 횟수
    - 총 임신 횟수 = IVF 임신 횟수 + DI 임신 횟수
    - 총 출산 횟수 = IVF 출산 횟수 + DI 출산 횟수
    - 변환된 column 개수: 3개
    - 변환된 column 목록: ['총 시술 횟수', '총 임신 횟수', '총 출산 횟수']
4. **나이 변환**
    -  median_age = True 일 경우 1) 시술 당시 나이, 2) 난자 기증자 나이, 3) 정자 기증자 나이 변수를 중앙값으로 변환
    -  median_age = False 일 경우 시술 당시 나이 중앙값 변수를 생성한 뒤 파생 변수 생성 이후 제거
5. **배아 생성 주요 이유 변환**
    - 배아 생성 주요 이유 변수를 시술용 여부 변수로 변환
6. **정자 출처 변환**
7. **파생 변수 생성**
    - 변수가 많아서 제거할 변수는 제거하고 대현님 분류에 맞게 다시 수정 후 정리 예정
8. **결측치 처리**
    - 현재 999로 대치하는 simple imputation 사용
    - 파생 변수 생성 과정에서 따로 값을 대치하는데 통일 필요
9. **범주형 변수 encoding**
10. **불필요한 열 추가 제거**
    - 중앙값 사용하지 않는 경우 '시술 당시 나이 중앙값 변수' 제거
    - 배아 생성 주요 이유, 파트너 정자와 혼합된 난자 수, 기증자 정자와 혼합된 난자 수 열 제거

Mapping Dict
- 시술 당시 나이, 난자 기증자 나이, 정자 기증자 나이 mapping dict 정의

In [3]:
PATIENT_MEDIAN_AGE_MAPPING = {
    '만18-34세': 26,  
    '만35-37세': 36,  
    '만38-39세': 38.5,  
    '만40-42세': 41,  
    '만43-44세': 43.5,  
    '만45-50세': 47.5,  
    '알 수 없음': np.nan
}

EGG_MEDIAN_AGE_MAPPING = {
    '만20세 이하': 18,  
    '만21-25세': 23,  
    '만26-30세': 28,  
    '만31-35세': 33,  
    '알 수 없음': np.nan
}

SPERM_MEDIAN_AGE_MAPPING = {
    '만20세 이하': 18,  
    '만21-25세': 23,
    '만26-30세': 28,
    '만31-35세': 33,  
    '만36-40세': 38,  
    '만41-45세': 43,  
    '알 수 없음': np.nan
}

AGE_MAPPING = {
    '만18-34세': 0,
    '만35-37세': 1,
    '만38-39세': 2,  
    '만40-42세': 3,
    '만43-44세': 4,
    '만45-50세': 5,
    '알 수 없음': np.nan
}

EGG_AGE_MAPPING = {
    '만20세 이하': 0,  
    '만21-25세': 1,
    '만26-30세': 2,  
    '만31-35세': 3,
    '알 수 없음': np.nan
}

SPERM_AGE_MAPPING = {
    '만20세 이하': 0,  
    '만21-25세': 1,
    '만26-30세': 2,
    '만31-35세': 3,  
    '만36-40세': 4,  
    '만41-45세': 5,  
    '알 수 없음': np.nan
}

전처리 관련 함수

In [4]:
def apply_preprocessing(
    data:pd.DataFrame,
    imputation_method:Literal['simple', 'knn'] = 'simple',
    median_age:bool = True,
) -> pd.DataFrame:
    # Step 1. col 제거
    # 결측 비율 80% 이상, nunique = 1, [배란 유도 유형, 특정 시술 유형, 남성 주 불임 원인, 남성 부 불임 원인, 여성 주 불임 원인, 여성 부 불임 원인, 부부 주 불임 원인, 부부 부 불임 원인] col 제거
    dropped_columns = list(set(
        data.isnull().mean()[data.isnull().mean() >= 0.8].index.tolist() + 
        data.nunique()[data.nunique() == 1].index.tolist() + 
        ['배란 유도 유형', '특정 시술 유형', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인']
    ))
    data.drop(columns = dropped_columns, axis = 1, inplace = True)
    print(f'제거된 컬럼: {dropped_columns}')
    print(f'제거된 컬럼 개수: {len(dropped_columns)}\n')
    
    # Step 2. int 변환
    count_columns = [col for col in data.columns if '횟수' in col]
    data[count_columns] = data[count_columns].apply(lambda x: x.str[0].astype(int))
    print(f'int 변환된 컬럼: {count_columns}')
    print(f'int 변환 컬럼 개수: {len(count_columns)}\n')
    
    # Step 3. 총 시술 횟수 재정의
    data['총 시술 횟수'] = data['IVF 시술 횟수'] + data['DI 시술 횟수']
    data['총 임신 횟수'] = data['IVF 임신 횟수'] + data['DI 임신 횟수']
    data['총 출산 횟수'] = data['IVF 출산 횟수'] + data['DI 출산 횟수']
    
    # Step 4. 나이 변환
    # median_age = True 일 경우 1) 시술 당시 나이, 2) 난자 기증자 나이, 3) 정자 기증자 나이 변수를 중앙값으로 변환
    # median_age = False 일 경우 시술 당시 나이 중앙값 변수를 생성한 뒤 파생 변수 생성 이후 제거
    if median_age:
        data['시술 당시 나이'] = data['시술 당시 나이'].map(PATIENT_MEDIAN_AGE_MAPPING)
        data['난자 기증자 나이'] = data['난자 기증자 나이'].map(EGG_MEDIAN_AGE_MAPPING)
        data['정자 기증자 나이'] = data['정자 기증자 나이'].map(SPERM_MEDIAN_AGE_MAPPING)
    else:
        data['시술 당시 나이 중앙값'] = data['시술 당시 나이'].map(PATIENT_MEDIAN_AGE_MAPPING)
        data['시술 당시 나이'] = data['시술 당시 나이'].map(AGE_MAPPING)
        data['난자 기증자 나이'] = data['난자 기증자 나이'].map(EGG_AGE_MAPPING)
        data['정자 기증자 나이'] = data['정자 기증자 나이'].map(SPERM_AGE_MAPPING)
        
    # Step 5. 배아 생성 주요 이유 변환
    data['시술용 여부'] = data['배아 생성 주요 이유'].astype(str).apply(lambda x: 1 if '현재 시술용' in x else 0)
    
    # Step 6. 정자 출처 변환
    change_count = ((data['정자 출처'] == '미할당') & (data['정자 기증자 나이'] != '알 수 없음')).sum()
    data.loc[(data['정자 출처'] == '미할당') & (data['정자 기증자 나이'] != '알 수 없음'), '정자 출처'] = '기증 제공'
    print(f"변경된 정자 출처 변수 개수: {change_count}개\n")
    
    # Step 7. 파생 변수 생성
    data = feature_engineering(data, median_age)
    
    # Step 8. 결측값 처리
    data = apply_imputation(data, imputation_method)
    
    # Step 9. 범수형 변수 encoding
    cat_features = list(data.select_dtypes(include = ['object']).columns)
    data[cat_features] = data[cat_features].astype(str).apply(lambda x: LabelEncoder().fit_transform(x))
    print(f'변환된 범주형 변수: {cat_features}\n')

    # Step 10. 불필요한 열 추가 제거
    if not median_age:
        data.drop('시술 당시 나이 중앙값',axis = 1, inplace = True)
    data.drop(['배아 생성 주요 이유', '파트너 정자와 혼합된 난자 수', "기증자 정자와 혼합된 난자 수"], axis = 1, inplace = True)
    
    return data

def feature_engineering(data:pd.DataFrame, median_age:bool) -> pd.DataFrame:
    # V1. 임신 성공률
    data['총 임신 성공률'] = data['총 임신 횟수'] / data['총 시술 횟수']
    data['IVF 임신 성공률'] = data['IVF 임신 횟수'] / data['IVF 시술 횟수']
    data['DI 임신 성공률'] = data['DI 임신 횟수'] / data['DI 시술 횟수']
    data['총 임신 성공률'] = handle_inf_and_fillna(data['총 임신 성공률'], fill_value = 0.5) # 결측값 처리 방법 통일 및 대체 값에 대한 논의 필요
    data['IVF 임신 성공률'] = handle_inf_and_fillna(data['IVF 임신 성공률'], fill_value = 0.5)
    data['DI 임신 성공률'] = handle_inf_and_fillna(data['DI 임신 성공률'], fill_value = 0.5)
    # data['임신 시도 대비 성공률'] = data['총 임신 횟수'] / (data['임신 시도 또는 마지막 임신 경과 연수'] + 1e-10) # 임신 시도 col 결측값으로 인해 col drop -> 사용할 예정이면 코드 변경 필요

    # V2. 출산 성공률
    data['총 출산 성공률'] = data['총 출산 횟수'] / data['총 임신 횟수']
    data['IVF 출산 성공률'] = data['IVF 출산 횟수'] / data['IVF 임신 횟수']
    data['DI 출산 성공률'] = data['DI 출산 횟수'] / data['DI 시술 횟수']
    data['배아 이식 대비 출산 성공률'] = data['총 출산 횟수'] / (data['이식된 배아 수'] + 1e-10)
    data['출산 유지율'] = data['총 출산 횟수'] / (data['총 임신 횟수'] + 1e-6)
    data['총 출산 성공률'] = handle_inf_and_fillna(data['총 출산 성공률'], fill_value = 0.5)
    data['IVF 출산 성공률'] = handle_inf_and_fillna(data['IVF 출산 성공률'], fill_value=0.5)
    data['DI 출산 성공률'] = handle_inf_and_fillna(data['DI 출산 성공률'], fill_value=0.5)
    
    # V3. 나이
    age_denominator = '시술 당시 나이' if median_age else '시술 당시 나이 중앙값'
    data['나이 대비 임신 확률'] = data['총 임신 횟수'] / data[age_denominator]
    data['나이 대비 배아 생성 확률'] = data['총 생성 배아 수'] / data[age_denominator]
    data['나이 대비 배아 이식 확률'] = data['이식된 배아 수'] / data[age_denominator]
    data['나이 대비 난자 생성 효율'] = data['수집된 신선 난자 수'] / data[age_denominator]
    data['나이 대비 임신 확률'] = handle_inf_and_fillna(data['나이 대비 임신 확률'], fill_value = 1)
    # data['나이 대비 임신 시도 기간'] = data['임신 시도 또는 마지막 임신 경과 연수'] / data[age_denominator] # 임신 시도 col 결측값으로 인해 col drop -> 사용할 예정이면 코드 변경 필요
    # data['나이 대비 임신 시도 기간'] = handle_inf_and_fillna(data['나이 대비 임신 시도 기간'], fill_value = 1)
    
    # V4. 시술 효율성
    data['시술당 평균 배아 생성 수'] = data['총 생성 배아 수'] / data['총 시술 횟수']
    data['시술당 평균 이식 배아 수'] = data['이식된 배아 수'] / data['총 시술 횟수'] 
    data['미세주입 배아 이식 확률'] = data['미세주입 배아 이식 수'] / (data['미세주입에서 생성된 배아 수'] + 1e-10)
    data['총 배아 이식 확률'] = data['이식된 배아 수'] / (data['총 생성 배아 수'] + 1e-10)
    data['시술당 평균 배아 생성 수'] = handle_inf_and_fillna(data['시술당 평균 배아 생성 수'], fill_value = 0)
    data['시술당 평균 이식 배아 수'] = handle_inf_and_fillna(data['시술당 평균 이식 배아 수'], fill_value = 0)
    
    # V5. 이식 경과일
    data["배아 이식 경과 카테고리"] = pd.cut(data["배아 이식 경과일"],
                             bins=[0, 2, 4, data["배아 이식 경과일"].max()], 
                             labels=[0, 1, 2])
    data["배아 이식 경과 카테고리"] = data["배아 이식 경과 카테고리"].astype(str)
    data['배아 저장 대비 이식 기간'] = data['배아 이식 경과일'] / (data['저장된 배아 수'] + 1e-10)
    data['이식된 배아 대비 이식 기간'] = data['배아 이식 경과일'] / (data['이식된 배아 수'] + 1e-10)
    data['배아 이식 경과*출산 성공률'] = data['배아 이식 경과일'] * data['배아 이식 대비 출산 성공률']
    
    # V6. 난자
    data['나이 대비 난자 생성 효율'] = data['수집된 신선 난자 수'] / data[age_denominator]
    data['총 난자 수'] = data['수집된 신선 난자 수'] + data['해동 난자 수']
    data['IVF 난자 수'] = data['혼합된 난자 수'] - data['미세주입된 난자 수']
    data['난자 사용률'] = data['혼합된 난자 수'] / data['총 난자 수']
    data['난자 혼합 비율'] = data['혼합된 난자 수'] / (data['수집된 신선 난자 수'] + 1e-10)
    # data['난자 채취-이식 경과 비율'] = data['배아 이식 경과일'] / (data['난자 채취 경과일'] + 1e-10) # 난자 채취 경과일 결측값으로 인해 col drop -> 사용할 예정이면 코드 변경 필요
    
    # V7. 배아
    data['총 배아 수'] = data['총 생성 배아 수'] + data['해동된 배아 수']
    data['IVF 배아 수'] = data['총 생성 배아 수'] - data['미세주입에서 생성된 배아 수']
    data['미세주입 배아 생성 확률'] = data['미세주입에서 생성된 배아 수'] / (data['미세주입된 난자 수'] + 1e-6)
    data['IVF 배아 생성 확률'] = data['IVF 배아 수'] / (data['IVF 난자 수'])
    data['총 배아 생성 확률']=data['총 배아 수']/(data['총 난자 수'])
    data['이식률'] = data['이식된 배아 수'] / (data['총 배아 수']) # Baseline의 V4. 의 총 배아 이식 확률과 차이점이 있는지 확인 후 통합 필요 -> 덕현님
    data['동결 배아 사용 비율'] = data['동결 배아 사용 여부'] / (data['동결 배아 사용 여부'] + data['신선 배아 사용 여부'] + 1e-10)
    data['동결&IVF 배아 이식 확률'] = (data['이식된 배아 수'] - data['미세주입 배아 이식 수']) / (data['총 배아 수'] - data['미세주입에서 생성된 배아 수'])
    data['난자 채취 대비 배아 생성 확률'] = data['총 생성 배아 수'] / (data['수집된 신선 난자 수'] + 1e-10)
    data['난자 채취 대비 배아 생성 기간'] = data['배아 이식 경과일'] / (data['수집된 신선 난자 수'] + 1e-6)
    data['저장된 배아 사용률'] = data['이식된 배아 수'] / (data['저장된 배아 수'] + 1e-10)
    data['해동 배아 사용률'] = data['해동된 배아 수'] / (data['저장된 배아 수'] + 1e-10)
    data["배아 저장 대비 이식 기간"] = data["배아 이식 경과일"] / (data["저장된 배아 수"] + 1e-10)
    # data['배아 해동 이후 이식 기간'] = data['배아 이식 경과일'] - data['배아 해동 경과일'] # 배아 해동 경과일 결측값으로 인해 col drop -> 사용할 예정이면 코드 변경 필요
    
    # V8. 임신, 출산 성공률 / 분모가 0인 경우 10으로 대체 -> 다른 컬럼의 결측치(unknown)과 비교
    data['총 시술 대비 임신 성공률'] = data['총 임신 횟수'] / data['총 시술 횟수'].replace(0, np.nan).fillna(10) # Baseline에서 제공받은 V1.의 총 임신 성공률과 다른점이 무엇인지 확인 후 통합 필요-> 나경님
    data['총 시술 대비 출산 성공률'] = data['총 출산 횟수'] / data['총 시술 횟수'].replace(0, np.nan).fillna(10)
    data['총 임신 대비 출산 성공률'] = data['총 출산 횟수'] / data['총 임신 횟수'].replace(0, np.nan).fillna(10) # Baseline에서 제공받은 V2.의 총 출산 성공률과 다른점이 무엇인지 확인 후 통합 필요-> 나경님

    data['IVF 시술 대비 임신 성공률'] = data['IVF 임신 횟수'] / data['IVF 시술 횟수'].replace(0, np.nan).fillna(10) # Baseline에서 제공받은 V1.의 IVF 임신 성공률과 다른점이 무엇인지 확인 후 통합 필요-> 나경님
    data['IVF 시술 대비 출산 성공률'] = data['IVF 출산 횟수'] / data['IVF 시술 횟수'].replace(0, np.nan).fillna(10)
    data['IVF 임신 대비 출산 성공률'] = data['IVF 출산 횟수'] / data['IVF 임신 횟수'].replace(0, np.nan).fillna(10) # Baseline에서 제공받은 V2.의 IVF 임신 대비 출산 성공률과 다른점이 무엇인지 확인 후 통합 필요-> 나경님

    data['DI 시술 대비 임신 성공률'] = data['DI 임신 횟수'] / data['DI 시술 횟수'].replace(0, np.nan).fillna(10)
    data['DI 시술 대비 출산 성공률'] = data['DI 출산 횟수'] / data['DI 시술 횟수'].replace(0, np.nan).fillna(10)
    data['DI 임신 대비 출산 성공률'] = data['DI 출산 횟수'] / data['DI 임신 횟수'].replace(0, np.nan).fillna(10)

    # V9. 나이 그룹별 평균값 / 분모가 0인 경우 10으로 대체 -> 다른 컬럼의 결측치(unknown)과 비교
    data['나이 그룹별 평균 시술 확률'] = data.groupby(age_denominator)['총 시술 횟수'].transform('mean').fillna(10)
    data['나이 그룹별 평균 임신 확률'] = data.groupby(age_denominator)['총 임신 횟수'].transform('mean').fillna(10)
    data['나이 그룹별 평균 출산 확률'] = data.groupby(age_denominator)['총 출산 횟수'].transform('mean').fillna(10)
    data.loc[data['시술 유형'] == 'IVF', '나이 그룹별 평균 생성 배아 수'] = data.groupby(age_denominator)['총 생성 배아 수'].transform('mean')
    data.loc[data['시술 유형'] == 'IVF', '나이 그룹별 평균 이식 배아 수'] = data.groupby(age_denominator)['이식된 배아 수'].transform('mean')
           
    # V10. IVF 시술 관련 계산 / 분모가 0인 경우 10으로 대체 -> 다른 컬럼의 결측치(unknown)과 비교
    data.loc[data['시술 유형'] == 'IVF', '시술당 평균 배아 생성 수'] = data['총 생성 배아 수'] / data['총 시술 횟수'].replace(0, np.nan).fillna(10)
    data.loc[data['시술 유형'] == 'IVF', '시술당 평균 이식 배아 수'] = data['이식된 배아 수'] / data['총 시술 횟수'].replace(0, np.nan).fillna(10)
    data.loc[data['시술 유형'] == 'IVF', '총 배아 생성 확률'] = data['총 생성 배아 수'] / data['총 시술 횟수'].replace(0, np.nan).fillna(10)
    data.loc[data['시술 유형'] == 'IVF', '총 배아 이식 확률'] = data['이식된 배아 수'] / data['총 생성 배아 수'].replace(0, np.nan).fillna(10)

    data.loc[data['시술 유형'] == 'IVF', '미세주입 배아 생성 확률'] = data['미세주입에서 생성된 배아 수'] / data['미세주입된 난자 수'].replace(0, np.nan).fillna(10)
    data.loc[data['시술 유형'] == 'IVF', '미세주입 배아 이식 확률'] = data['미세주입 배아 이식 수'] / data['미세주입에서 생성된 배아 수'].replace(0, np.nan).fillna(10)
    
    data.loc[data['시술 유형'] == 'IVF', '배아 이식 대비 임신 성공률'] = data['총 임신 횟수'] / data['이식된 배아 수'].replace(0, np.nan).fillna(10)
    data.loc[data['시술 유형'] == 'IVF', '배아 이식 대비 출산 성공률'] = data['총 출산 횟수'] / data['이식된 배아 수'].replace(0, np.nan).fillna(10)
    
    # V11. 시술
    data['시술 경험 점수'] = (data['총 시술 횟수'] + data['클리닉 내 총 시술 횟수']) / 2
    data['시술 시기별 성공률'] = data.groupby('시술 시기 코드')['총 임신 횟수'].transform('mean') / (data.groupby('시술 시기 코드')['총 시술 횟수'].transform('mean') + 1e-10)
    
    # V12. 불임 원인
    if '불임 원인 - 자궁내막증' in data.columns and '불임 원인 - 자궁경부 문제' in data.columns:
        data['여성 불임 세부 지표'] = data['불임 원인 - 자궁내막증'] + data['불임 원인 - 자궁경부 문제']

    if ('불임 원인 - 정자 농도' in data.columns and 
        '불임 원인 - 정자 운동성' in data.columns and 
        '불임 원인 - 정자 형태' in data.columns):
        data['남성 정자 종합 지표'] = (
            data['불임 원인 - 정자 농도'] 
            + data['불임 원인 - 정자 운동성'] 
            + data['불임 원인 - 정자 형태']
        )
        
    # 특정 시술 유형 사용 여부 논의 필요 -> Baseline에서는 제거
    # data['최고 성공률 시술 유형'] = data[['특정 시술 유형 - IVF', '특정 시술 유형 - ICSI', '특정 시술 유형 - IUI']].idxmax(axis = 0)
    # data['시술 유형 성공 확률'] = data[['특정 시술 유형 - IVF', '특정 시술 유형 - ICSI', '특정 시술 유형 - IUI']].mean(axis = 0)
    
    return data


def apply_imputation(
    data:pd.DataFrame,
    imputation_method:Literal['simple', 'knn'],
    fill_value:int = 999,
) -> pd.DataFrame:
    if imputation_method == 'simple':
        data.replace([np.inf, -np.inf], np.nan, inplace = True)
        data.fillna(fill_value, inplace = True)
    
    elif imputation_method == 'knn':
        data.replace([np.inf, -np.inf], np.nan, inplace = True)
        imputer = KNNImputer(n_neighbors = 5)
        data = pd.DataFrame(imputer.fit_transform(data), columns = data.columns)
    
    return data


def handle_inf_and_fillna(data, fill_value = 0):
    """
    DataFrame data 내부에 inf, -inf가 있으면 np.nan으로 치환 후,
    그 NaN 값을 fill_value(기본=0)로 대체한다.
    inplace=False로 처리해 새로운 DataFrame을 반환.
    """
    # 1) inf -> NaN 치환
    data_replaced = data.replace([np.inf, -np.inf], np.nan)

    # 2) 결측치(NaN)를 fill_value로 대체
    data_filled = data_replaced.fillna(fill_value)

    return data_filled

데이터 불러오기 및 전처리 함수 적용

In [5]:
df_train = pd.read_csv('./data/train.csv').drop(columns = ['ID'])
df_test = pd.read_csv('./data/test.csv').drop(columns = ['ID'])

df_train = apply_preprocessing(df_train)
df_test = apply_preprocessing(df_test)

제거된 컬럼: ['PGS 시술 여부', '착상 전 유전 검사 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '특정 시술 유형', '배아 해동 경과일', '여성 주 불임 원인', '임신 시도 또는 마지막 임신 경과 연수', '배란 유도 유형', '여성 부 불임 원인', '부부 부 불임 원인', '난자 채취 경과일', '난자 해동 경과일', '부부 주 불임 원인', 'PGD 시술 여부', '불임 원인 - 여성 요인']
제거된 컬럼 개수: 16

int 변환된 컬럼: ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']
int 변환 컬럼 개수: 10

변경된 정자 출처 변수 개수: 122개

변환된 범주형 변수: ['시술 시기 코드', '시술 유형', '배아 생성 주요 이유', '난자 출처', '정자 출처', '배아 이식 경과 카테고리']

제거된 컬럼: ['PGS 시술 여부', '착상 전 유전 검사 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '특정 시술 유형', '배아 해동 경과일', '여성 주 불임 원인', '임신 시도 또는 마지막 임신 경과 연수', '배란 유도 유형', '여성 부 불임 원인', '부부 부 불임 원인', '난자 채취 경과일', '난자 해동 경과일', '부부 주 불임 원인', 'PGD 시술 여부', '불임 원인 - 여성 요인']
제거된 컬럼 개수: 16

int 변환된 컬럼: ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']
int 변환 컬럼 개수: 10

변경된 정자 출처 변수 개수: 33개

변환된 범주형 변수: ['시술 시기 코드', '시술 유형', '배아 생성

결측값 확인

In [6]:
df_train.isnull().sum()

시술 시기 코드              0
시술 당시 나이              0
시술 유형                 0
배란 자극 여부              0
단일 배아 이식 여부           0
착상 전 유전 진단 사용 여부      0
불명확 불임 원인             0
불임 원인 - 난관 질환         0
불임 원인 - 남성 요인         0
불임 원인 - 배란 장애         0
불임 원인 - 자궁경부 문제       0
불임 원인 - 자궁내막증         0
불임 원인 - 정자 농도         0
불임 원인 - 정자 면역학적 요인    0
불임 원인 - 정자 운동성        0
불임 원인 - 정자 형태         0
총 시술 횟수               0
클리닉 내 총 시술 횟수         0
IVF 시술 횟수             0
DI 시술 횟수              0
총 임신 횟수               0
IVF 임신 횟수             0
DI 임신 횟수              0
총 출산 횟수               0
IVF 출산 횟수             0
DI 출산 횟수              0
총 생성 배아 수             0
미세주입된 난자 수            0
미세주입에서 생성된 배아 수       0
이식된 배아 수              0
미세주입 배아 이식 수          0
저장된 배아 수              0
미세주입 후 저장된 배아 수       0
해동된 배아 수              0
해동 난자 수               0
수집된 신선 난자 수           0
저장된 신선 난자 수           0
혼합된 난자 수              0
난자 출처                 0
정자 출처                 0
난자 기증자 나이             0
정자 기증자 나이       

In [7]:
df_test.isnull().sum()

시술 시기 코드              0
시술 당시 나이              0
시술 유형                 0
배란 자극 여부              0
단일 배아 이식 여부           0
착상 전 유전 진단 사용 여부      0
불명확 불임 원인             0
불임 원인 - 난관 질환         0
불임 원인 - 남성 요인         0
불임 원인 - 배란 장애         0
불임 원인 - 자궁경부 문제       0
불임 원인 - 자궁내막증         0
불임 원인 - 정자 농도         0
불임 원인 - 정자 면역학적 요인    0
불임 원인 - 정자 운동성        0
불임 원인 - 정자 형태         0
총 시술 횟수               0
클리닉 내 총 시술 횟수         0
IVF 시술 횟수             0
DI 시술 횟수              0
총 임신 횟수               0
IVF 임신 횟수             0
DI 임신 횟수              0
총 출산 횟수               0
IVF 출산 횟수             0
DI 출산 횟수              0
총 생성 배아 수             0
미세주입된 난자 수            0
미세주입에서 생성된 배아 수       0
이식된 배아 수              0
미세주입 배아 이식 수          0
저장된 배아 수              0
미세주입 후 저장된 배아 수       0
해동된 배아 수              0
해동 난자 수               0
수집된 신선 난자 수           0
저장된 신선 난자 수           0
혼합된 난자 수              0
난자 출처                 0
정자 출처                 0
난자 기증자 나이             0
정자 기증자 나이       

## 학습 및 검증 데이터 분할

In [8]:
X = df_train.drop('임신 성공 여부', axis = 1)
y = df_train['임신 성공 여부']

# SEED = 123
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, stratify = y, shuffle = True, random_state = SEED)
X_test = df_test

## Sampling
- 추후 적용

In [9]:
# def apply_sampling(
#     X: pd.DataFrame,
#     y: pd.Series,
#     sampling_method: Optional[Literal['smote', 'under_sampling', 'smote_tomek', 'smote_enn']] = None,
#     random_state: int = 0
# ) -> pd.DataFrame:
    
#     if sampling_method == 'smote':
#         smote = SMOTE(random_state = random_state)
#         X_resampled, y_resampled = smote.fit_resample(X, y)
        
#     elif sampling_method == 'under_sampling':
#         under_sampler = RandomUnderSampler(random_state=random_state)
#         X_resampled, y_resampled = under_sampler.fit_resample(X, y)
    
#     elif sampling_method == 'smote_tomek':
#         smote_tomek = SMOTETomek(random_state = random_state)
#         X_resampled, y_resampled = smote_tomek.fit_resample(X, y)
        
#     elif sampling_method == 'smote_enn':
#         smote_enn = SMOTEENN(random_state = random_state)
#         X_resampled, y_resampled = smote_enn.fit_resample(X, y)
        
#     else:
#         X_resampled, y_resampled = X, y
        
#     print(f"Original dataset shape: {dict(pd.Series(y).value_counts())}")
#     print(f"Resampled dataset shape: {dict(pd.Series(y_resampled).value_counts())}")
    
#     return X_resampled, y_resampled

## Scaling
- 추후 적용

In [10]:
# def apply_scaling(
#     data:pd.DataFrame,
#     scaling_columns:List[str],
#     scaling_method:Literal['MinMax', 'Standard']
# ) -> pd.DataFrame:
#     pass

# scaling_columns = [
#     '총 생성 배아 수',
#     '미세주입된 난자 수',
#     '미세주입에서 생성된 배아 수',
#     '이식된 배아 수',
#     '미세주입 배아 이식 수', 
#     '저장된 배아 수',
#     '미세주입 후 저장된 배아 수',
#     '해동된 배아 수',
#     '해동 난자 수',
#     '수집된 신선 난자 수',
#     '저장된 신선 난자 수',
#     '혼합된 난자 수',
#     '파트너 정자와 혼합된 난자 수',
#     '기증자 정자와 혼합된 난자 수'
# ]

# 모델 학습
- 함수 변환 및 수정은 추후 적용
- 경고 관련 내용 수정 예정
- X_train, y_train만 사용해 Cross Validation 진행
- X_val, y_val은 테스트 데이터 처럼 사용

In [11]:
# Stratified K-Fold 설정
n_splits = 5
skf = StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = SEED)

metrics = {model: [] for model in ['CatBoost', 'XGBoost', 'LightGBM', 'AdaBoost', 'Ensemble']}
feature_importances = {model: [] for model in ['CatBoost', 'XGBoost', 'LightGBM', 'AdaBoost']}
test_proba = {model: [] for model in ['CatBoost', 'XGBoost', 'LightGBM', 'AdaBoost', 'Ensemble']}

# 
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train), 1):
    print(f"===== Fold {fold} =====")

    X_train_temp, X_val_temp = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_temp, y_val_temp = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # 모델 정의
    cat_model = CatBoostClassifier(
        iterations=700, learning_rate=0.03, depth=8, l2_leaf_reg=10,
        subsample=0.8, colsample_bylevel=0.8, random_strength=10,
        loss_function='Logloss', eval_metric='AUC', verbose=100
    )

    xgb_model = XGBClassifier(
        n_estimators=700, learning_rate=0.03, max_depth=7, min_child_weight=3,
        gamma=0.1, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.1,
        reg_lambda=1.0, verbosity=1
    )

    lgbm_model = LGBMClassifier(
        n_estimators=700, learning_rate=0.03, max_depth=-1, num_leaves=64,
        min_child_samples=20, subsample=0.8, colsample_bytree=0.8,
        reg_alpha=0.1, reg_lambda=1.0, verbosity=1
    )
    
    adaboost_model = AdaBoostClassifier(
        estimator = DecisionTreeClassifier(max_depth=2, min_samples_split=10, min_samples_leaf=5, random_state = SEED),
        n_estimators=500, learning_rate=0.05,
        algorithm="SAMME", random_state = SEED
    )
    
    ensemble_model = VotingClassifier(
        estimators=[('catboost', cat_model), ('xgboost', xgb_model), ('lightgbm', lgbm_model), ('adaboost', adaboost_model)],
        voting='soft', weights=[1, 1, 1, 0.8]
    )

    # 모델 학습
    for model in [cat_model, xgb_model, lgbm_model, adaboost_model, ensemble_model]:
        model.fit(X_train_temp, y_train_temp)

    # 평가 함수
    def evaluate_model(model, X_val, y_true):
        y_pred = model.predict(X_val)
        y_pred_proba = model.predict_proba(X_val)[:, 1]

        return {
            'Accuracy': accuracy_score(y_true, y_pred),
            'Precision': precision_score(y_true, y_pred),
            'Recall': recall_score(y_true, y_pred),
            'F1 Score': f1_score(y_true, y_pred),
            'ROC AUC Score': roc_auc_score(y_true, y_pred_proba)
        }

    # 평가 및 변수 중요도 저장
    for model_name, model in zip(metrics.keys(), [cat_model, xgb_model, lgbm_model, adaboost_model, ensemble_model]):
        metrics[model_name].append(evaluate_model(model, X_val_temp, y_val_temp))

    for model_name, model in zip(['CatBoost', 'XGBoost', 'LightGBM', 'AdaBoost'], [cat_model, xgb_model, lgbm_model, adaboost_model]):
        feature_importances[model_name].append(model.feature_importances_)
        
    # 테스트 데이터 예측 확률 저장
    test_proba['CatBoost'].append(cat_model.predict_proba(X_test)[:, 1])
    test_proba['XGBoost'].append(xgb_model.predict_proba(X_test)[:, 1])
    test_proba['LightGBM'].append(lgbm_model.predict_proba(X_test)[:, 1])
    test_proba['AdaBoost'].append(adaboost_model.predict_proba(X_test)[:, 1])
    test_proba['Ensemble'].append(ensemble_model.predict_proba(X_test)[:, 1])
  


===== Fold 1 =====
0:	total: 153ms	remaining: 1m 46s
100:	total: 2.6s	remaining: 15.4s
200:	total: 5.13s	remaining: 12.7s
300:	total: 7.7s	remaining: 10.2s
400:	total: 10.2s	remaining: 7.62s
500:	total: 12.9s	remaining: 5.11s
600:	total: 15.4s	remaining: 2.54s
699:	total: 17.8s	remaining: 0us


Exception in thread Thread-3 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\ANDLab001\anaconda3\envs\aimers\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\ANDLab001\anaconda3\envs\aimers\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\ANDLab001\anaconda3\envs\aimers\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\ANDLab001\anaconda3\envs\aimers\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\ANDLab001\anaconda3\envs\aimers\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 24: invalid start byte
  File "c:\Users\ANDLab001\anaconda3\envs\aimers\lib\site-packages\joblib\externals\loky\backend\context.py", line 262, in _count_physical_cores
    cpu

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 42386, number of negative: 121678
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011144 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3483
[LightGBM] [Info] Number of data points in the train set: 164064, number of used features: 102
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258350 -> initscore=-1.054560
[LightGBM] [Info] Start training from score -1.054560
0:	total: 24.8ms	remaining: 17.4s
100:	total: 2.53s	remaining: 15s
200:	total: 4.94s	remaining: 12.3s
300:	total: 7.25s	remaining: 9.61s
400:	total: 9.65s	remaining: 7.2s
500:	total: 12.2s	remaining: 4.86s
600:	total: 14.9s	remaining: 2.45s
699:	total: 17.4s	remaining: 0us
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[Ligh

Validation data AUC ROC Score

In [17]:
# 평가 함수
def evaluate_model(model, X_val, y_true):
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred), 
        'Recall': recall_score(y_true, y_pred),
        'F1 Score': f1_score(y_true, y_pred),
        'ROC AUC Score': roc_auc_score(y_true, y_pred_proba)
    }

# 검증 데이터에 대한 성능 평가
print("\n===== 검증 데이터 성능 =====")
for model_name, model in zip(metrics.keys(), [cat_model, xgb_model, lgbm_model, adaboost_model, ensemble_model]):
    val_metrics = evaluate_model(model, X_val, y_val)
    print(f"\n== {model_name} Model ==")
    for metric, value in val_metrics.items():
        print(f"{metric}: {value:.6f}")


===== 검증 데이터 성능 =====

== CatBoost Model ==
Accuracy: 0.745821
Precision: 0.541345
Recall: 0.105768
F1 Score: 0.176961
ROC AUC Score: 0.739967

== XGBoost Model ==
Accuracy: 0.744456
Precision: 0.519978
Recall: 0.141477
F1 Score: 0.222433
ROC AUC Score: 0.737712

== LightGBM Model ==
Accuracy: 0.744631
Precision: 0.523175
Recall: 0.130379
F1 Score: 0.208739
ROC AUC Score: 0.737173

== AdaBoost Model ==
Accuracy: 0.743754
Precision: 0.530612
Recall: 0.070663
F1 Score: 0.124717
ROC AUC Score: 0.733818

== Ensemble Model ==
Accuracy: 0.745899
Precision: 0.537406
Recall: 0.118224
F1 Score: 0.193812
ROC AUC Score: 0.739630


평가 지표 평균 출력

In [18]:
print("===== Stratified K-Fold 평균 성능 =====")
for model_name, model_metrics in metrics.items():
    avg_metrics = {metric: np.mean([fold_metric[metric] for fold_metric in model_metrics]) for metric in model_metrics[0]}
    
    print(f"\n== {model_name} Model ==")
    for metric, value in avg_metrics.items():
        print(f"{metric}: {value:.6f}")

===== Stratified K-Fold 평균 성능 =====

== CatBoost Model ==
Accuracy: 0.745402
Precision: 0.535928
Recall: 0.108924
F1 Score: 0.181000
ROC AUC Score: 0.739262

== XGBoost Model ==
Accuracy: 0.744534
Precision: 0.520259
Recall: 0.143822
F1 Score: 0.225328
ROC AUC Score: 0.736724

== LightGBM Model ==
Accuracy: 0.744734
Precision: 0.523422
Recall: 0.133404
F1 Score: 0.212598
ROC AUC Score: 0.736811

== AdaBoost Model ==
Accuracy: 0.744441
Precision: 0.540777
Recall: 0.071779
F1 Score: 0.126728
ROC AUC Score: 0.733748

== Ensemble Model ==
Accuracy: 0.745494
Precision: 0.533347
Recall: 0.119512
F1 Score: 0.195237
ROC AUC Score: 0.739044


# Prediction
- 모델 성능 평가 후 가장 높은 AUC를 가진 모델 선택후 pred_proba 출력

In [19]:
model_aucs = {}
for model_name, model_metrics in metrics.items():
    avg_auc = np.mean([fold_metric['ROC AUC Score'] for fold_metric in model_metrics])
    model_aucs[model_name] = avg_auc

best_model = max(model_aucs.items(), key=lambda x: x[1])
print(f"\n최고 성능 모델: {best_model[0]} (ROC AUC: {best_model[1]:.6f})")

pred_proba = np.mean(test_proba[best_model[0]], axis = 0)


최고 성능 모델: CatBoost (ROC AUC: 0.739262)


In [14]:
# # Best AUC 기록한 모델의 pred_proba로 선택
# pred_proba = np.mean(test_proba['CatBoost'], axis = 0)
# # pred_proba = np.mean(test_proba['XGBoost'], axis=0)
# # pred_proba = np.mean(test_proba['LightGBM'], axis=0)
# # pred_proba = np.mean(test_proba['AdaBoost'], axis=0)
# # pred_proba = np.mean(test_proba['Ensemble'], axis=0)

# Submission


In [20]:
sample_submission_path = './data/sample_submission.csv'
sample_submission = pd.read_csv(sample_submission_path)
sample_submission.head()

,ID,probability
0,TEST_00000,0.0
1,TEST_00001,0.0
2,TEST_00002,0.0
3,TEST_00003,0.0
4,TEST_00004,0.0


In [21]:
sample_submission['probability'] = pred_proba

# 저장
submission_path = './data/submission.csv'
sample_submission.to_csv(submission_path, index = False)
sample_submission.head()

,ID,probability
0,TEST_00000,0.002205
1,TEST_00001,0.001162
2,TEST_00002,0.143161
3,TEST_00003,0.109168
4,TEST_00004,0.506670


In [22]:
# 확인용
submission = pd.read_csv(submission_path)
submission.head()

,ID,probability
0,TEST_00000,0.002205
1,TEST_00001,0.001162
2,TEST_00002,0.143161
3,TEST_00003,0.109168
4,TEST_00004,0.506670
